# Predicting US Movies Gross Using Regression (Scraping)

<blockquote> In this Jupyter notebook, we will scrape two websites, The Numbers and the IMDb. Accordingly, the Numbers website is used to get the movie's names to search them in the IMDb to get the data.</blockquote>

In [214]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from time import sleep
from random import randint

***

In [5]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

#### The Numbers website

In [288]:
url = "https://www.the-numbers.com/movie/budgets/all"
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")

In [289]:
page_num = ['',101,201,301,401,501, 601, 701, 801, 901, 1000]
df_TheNumbers = pd.DataFrame()
for page in page_num:
    if page == '':
        url = "https://www.the-numbers.com/movie/budgets/all"
    else:
        url = "https://www.the-numbers.com/movie/budgets/all/"+str(page)
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page)
    table_tag = soup.find("table")
    tr_data = table_tag.find_all("tr")
    movie_table = pd.DataFrame()
    for j in range(len(tr_data)):
        if j==0:
            table_tag_data = tr_data[j].find_all('th')
        else:
            table_tag_data = tr_data[j].find_all('td')
        movie_data = []
        for i in table_tag_data:
            movie_data.append(i.getText())
        movie_table = movie_table.append(pd.DataFrame([movie_data]))
    movie_table.columns = movie_table.iloc[0]
    movie_table = movie_table[1:]
    df_TheNumbers = df_TheNumbers.append(movie_table)
df_TheNumbers = df_TheNumbers.reset_index(drop=True)

In [290]:
df_TheNumbers.head()

,,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,1,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
2,3,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
3,4,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,064,615,817"
4,5,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,044,540,523"


In [291]:
df_TheNumbers = df_TheNumbers[["ReleaseDate", "Movie", "ProductionBudget", "DomesticGross", "WorldwideGross"]]

In [292]:
df_TheNumbers

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,064,615,817"
4,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,044,540,523"
...,...,...,...,...,...
1095,"Dec 25, 2008",The Spirit,"$60,000,000","$19,806,188","$39,006,188"
1096,"Oct 19, 2001",The Last Castle,"$60,000,000","$18,208,078","$20,541,668"
1097,"Jan 23, 2009",Inkheart,"$60,000,000","$17,303,424","$66,655,938"
1098,"Feb 18, 2020",Monster Hunter,"$60,000,000","$15,104,790","$44,400,541"


> **Note:** after exploring the data, we found an inconsistent data row. Therefore, we will handle it by drop the row out.

In [293]:
df_TheNumbers.loc[df_TheNumbers['ReleaseDate'] == 'Unknown']

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
798,Unknown,b,"$70,000,000",$0,$0


In [294]:
df_TheNumbers = df_TheNumbers.drop(df_TheNumbers.index[798])
df_TheNumbers

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
2,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
3,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,064,615,817"
4,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,044,540,523"
...,...,...,...,...,...
1095,"Dec 25, 2008",The Spirit,"$60,000,000","$19,806,188","$39,006,188"
1096,"Oct 19, 2001",The Last Castle,"$60,000,000","$18,208,078","$20,541,668"
1097,"Jan 23, 2009",Inkheart,"$60,000,000","$17,303,424","$66,655,938"
1098,"Feb 18, 2020",Monster Hunter,"$60,000,000","$15,104,790","$44,400,541"


In [295]:
df_TheNumbers.loc[df_TheNumbers_['ReleaseDate'] == 'Unknown']

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross


***
***Now we can Format the Release Date column into a readable date***
***

In [296]:
df_TheNumbers['ReleaseDate'] = pd.to_datetime(df_TheNumbers['ReleaseDate'], format='%b %d, %Y')
df_TheNumbers

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,2019-04-23,Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
2,2015-04-22,Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
3,2015-12-16,Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,064,615,817"
4,2018-04-25,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,044,540,523"
...,...,...,...,...,...
1095,2008-12-25,The Spirit,"$60,000,000","$19,806,188","$39,006,188"
1096,2001-10-19,The Last Castle,"$60,000,000","$18,208,078","$20,541,668"
1097,2009-01-23,Inkheart,"$60,000,000","$17,303,424","$66,655,938"
1098,2020-02-18,Monster Hunter,"$60,000,000","$15,104,790","$44,400,541"


***
***Now we Format both the ProductionBudget, DomesticGross and orldwideGross column into a readable date integer***
***

In [297]:
df_TheNumbers['ProductionBudget'] = df_TheNumbers['ProductionBudget'].str.replace('$','')
df_TheNumbers['DomesticGross'] = df_TheNumbers['DomesticGross'].str.replace('$','')
df_TheNumbers['WorldwideGross'] = df_TheNumbers['WorldwideGross'].str.replace('$','')

/Users/leenaabdullahqgmail.com/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/Users/leenaabdullahqgmail.com/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  
/Users/leenaabdullahqgmail.com/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so w

In [298]:
df_TheNumbers['ProductionBudget'] = df_TheNumbers['ProductionBudget'].str.replace(',','')
df_TheNumbers['DomesticGross'] = df_TheNumbers['DomesticGross'].str.replace(',','')
df_TheNumbers['WorldwideGross'] = df_TheNumbers['WorldwideGross'].str.replace(',','')

In [299]:
df_TheNumbers['ProductionBudget'] = df_TheNumbers['ProductionBudget'].astype(int)
df_TheNumbers['DomesticGross'] = df_TheNumbers['DomesticGross'].astype(int)
df_TheNumbers['WorldwideGross'] = df_TheNumbers['WorldwideGross'].astype(int)

In [300]:
df_TheNumbers

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,2019-04-23,Avengers: Endgame,400000000,858373000,2797800564
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,379000000,241071802,1045713802
2,2015-04-22,Avengers: Age of Ultron,365000000,459005868,1395316979
3,2015-12-16,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2064615817
4,2018-04-25,Avengers: Infinity War,300000000,678815482,2044540523
...,...,...,...,...,...
1095,2008-12-25,The Spirit,60000000,19806188,39006188
1096,2001-10-19,The Last Castle,60000000,18208078,20541668
1097,2009-01-23,Inkheart,60000000,17303424,66655938
1098,2020-02-18,Monster Hunter,60000000,15104790,44400541


In [301]:
df_TheNumbers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1099 entries, 0 to 1099
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ReleaseDate       1099 non-null   datetime64[ns]
 1   Movie             1099 non-null   object        
 2   ProductionBudget  1099 non-null   int64         
 3   DomesticGross     1099 non-null   int64         
 4   WorldwideGross    1099 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 51.5+ KB


In [210]:
df_TheNumbers.to_csv('The_Numbers_dataset.csv', encoding = 'utf-8', index=False)

***

#### The IMDb website

In [255]:
titles = []
years = []
duration = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []
genre = [] 
certificate = [] #

In [275]:
pages = np.arange(1, 3001, 50)
headers = {'Accept-Language': 'en-US, en;q=0.5'}

# Storing each of the urls of 50 movies 
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&countries=us&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year text-muted unbold').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        duration.append(runtime)
        
        # Scraping the certficate
        cert = container.find('span', class_='certificate').text if container.p.find('span', class_='certificate') else '-'
        certificate.append(cert)
        
        # Scraping the genere
        gen = container.find('span', class_='genre').text if container.p.find('span', class_='genre') else '-'
        genre.append(gen)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

In [276]:
movies = pd.DataFrame({'Movie':titles,
                       'year':years,
                       'time_minute':duration,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross,
                       'certificates': certificate,
                       'genres': genre})

movies.head()

,Movie,year,time_minute,imdb_rating,metascore,vote,gross_earning,certificates,genres
0,No Time to Die,(2021),163 min,7.6,68,"107,163",-,PG-13,"\nAction, Adventure, Thriller"
1,The Guilty,(2021),90 min,6.3,63,"64,375",-,R,"\nCrime, Drama, Thriller"
2,The Many Saints of Newark,(2021),120 min,6.4,60,"27,145",-,R,"\nCrime, Drama"
3,Venom: Let There Be Carnage,(2021),97 min,6.4,47,"30,443",-,PG-13,"\nAction, Adventure, Sci-Fi"
4,Dune,(2021),155 min,8.3,75,"84,636",-,PG-13,"\nAction, Adventure, Drama"


In [279]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Movie          6000 non-null   object 
 1   year           6000 non-null   object 
 2   time_minute    6000 non-null   object 
 3   imdb_rating    6000 non-null   float64
 4   metascore      6000 non-null   object 
 5   vote           6000 non-null   object 
 6   gross_earning  6000 non-null   object 
 7   certificates   6000 non-null   object 
 8   genres         6000 non-null   object 
dtypes: float64(1), object(8)
memory usage: 422.0+ KB


In [573]:
movies.iloc[0:3000]

,Movie,year,time_minute,imdb_rating,metascore,vote,gross_earning,certificates,genres
0,No Time to Die,(2021),163 min,7.6,68,"107,163",-,PG-13,"\nAction, Adventure, Thriller"
1,The Guilty,(2021),90 min,6.3,63,"64,375",-,R,"\nCrime, Drama, Thriller"
2,The Many Saints of Newark,(2021),120 min,6.4,60,"27,145",-,R,"\nCrime, Drama"
3,Venom: Let There Be Carnage,(2021),97 min,6.4,47,"30,443",-,PG-13,"\nAction, Adventure, Sci-Fi"
4,Dune,(2021),155 min,8.3,75,"84,636",-,PG-13,"\nAction, Adventure, Drama"
...,...,...,...,...,...,...,...,...,...
2995,J. Edgar,(2011),137 min,6.5,59,"125,594",$37.31M,R,"\nBiography, Drama, Romance"
2996,Happiness,(1998),134 min,7.7,81,"68,481",$2.81M,NC-17,"\nComedy, Drama"
2997,Dial M for Murder,(1954),105 min,8.2,75,"167,275",$0.01M,PG,"\nCrime, Thriller"
2998,Dear John,(I) (2010),108 min,6.3,43,"143,822",$80.01M,PG-13,"\nDrama, Mystery, Romance"


***
***Mergining The numbers dataframe and IMDb dataframe with matching movie name***
***

In [570]:
df_TheNumbers['year_date'] = pd.DatetimeIndex(df_TheNumbers['ReleaseDate']).year

In [579]:
df_new = pd.merge(movies, df_TheNumbers, how='inner', on = 'Movie')

In [564]:
df_new

,Movie,year,time_minute,imdb_rating,metascore,vote,gross_earning,certificates,genres,ReleaseDate,ProductionBudget,DomesticGross,WorldwideGross,Movie_year
0,No Time to Die,2021,163 min,7.6,68,"107,163",-,PG-13,"\nAction, Adventure, Thriller",2021-09-29,250000000,99509477,447809477,No Time to Die 2021
1,No Time to Die,2021,163 min,7.6,68,"107,370",-,PG-13,"\nAction, Adventure, Thriller",2021-09-29,250000000,99509477,447809477,No Time to Die 2021
2,Dune,2021,155 min,8.3,75,"84,636",-,PG-13,"\nAction, Adventure, Drama",2021-09-15,165000000,0,116751980,Dune 2021
3,Dune,1984,137 min,6.4,41,"142,157",$30.93M,PG-13,"\nAction, Adventure, Sci-Fi",2021-09-15,165000000,0,116751980,Dune 1984
4,Dune,2021,155 min,8.3,75,"84,924",-,PG-13,"\nAction, Adventure, Drama",2021-09-15,165000000,0,116751980,Dune 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,Surrogates,2009,89 min,6.3,45,"172,072",$38.58M,PG-13,"\nAction, Sci-Fi, Thriller",2009-09-25,80000000,38577772,119668350,Surrogates 2009
1526,Over the Hedge,2006,83 min,6.7,67,"166,838",$155.02M,PG,"\nAnimation, Adventure, Comedy",2006-05-19,80000000,155019340,343397247,Over the Hedge 2006
1527,Over the Hedge,2006,83 min,6.7,67,"166,838",$155.02M,PG,"\nAnimation, Adventure, Comedy",2006-05-19,80000000,155019340,343397247,Over the Hedge 2006
1528,The Manchurian Candidate,2004,129 min,6.6,76,"108,089",$65.96M,R,"\nDrama, Mystery, Sci-Fi",2004-07-30,80000000,65948711,96135872,The Manchurian Candidate 2004


***Now, let's clean the column "year" and combine it with the movie title.***

In [545]:
df_new['year'] = df_new['year'].str.strip('()').astype(str)
df_new['year'] =df_new['year'].astype(str).str.replace("I", "")
df_new['year'] =df_new['year'].astype(str).str.replace("(", "")
df_new['year'] =df_new['year'].astype(str).str.replace(")", "")

/Users/leenaabdullahqgmail.com/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/leenaabdullahqgmail.com/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


In [546]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1530 entries, 0 to 1529
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Movie             1530 non-null   object        
 1   year              1530 non-null   object        
 2   time_minute       1530 non-null   object        
 3   imdb_rating       1530 non-null   float64       
 4   metascore         1530 non-null   object        
 5   vote              1530 non-null   object        
 6   gross_earning     1530 non-null   object        
 7   certificates      1530 non-null   object        
 8   genres            1530 non-null   object        
 9   ReleaseDate       1530 non-null   datetime64[ns]
 10  ProductionBudget  1530 non-null   int64         
 11  DomesticGross     1530 non-null   int64         
 12  WorldwideGross    1530 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(8)
memory usage: 167.3+ KB


In [575]:
movies.to_csv('The_imdb_dataset_edited.csv', encoding = 'utf-8', index=False)
df_new_clean.to_csv('The_merged_edited.csv', encoding = 'utf-8', index=False)

### Conclusion

> We conclude that we only need the IMDb website in our project because there is no relation between the two websites.